<a href="https://colab.research.google.com/github/ssizawa/image-scraping/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import urllib.request
import re
from bs4 import BeautifulSoup

In [ ]:
n=94
furniture=[]
for i in range(n):
  if i==0:
    page_url='https://www.houzz.com/products/armchairs-and-accent-chairs/material_fabric--Polyester'
  else:
    page_url='https://www.houzz.com/products/armchairs-and-accent-chairs/material_fabric--Polyester/p/%s'%(i*36)

  # print(page_url)

  r = requests.get(page_url)

  soup = BeautifulSoup(r.text, 'html.parser')
  # soup.find_all('img', class_=' hz-image hz-responsive-img')
  img_tags=soup.find_all('a',  class_=re.compile('ProductTitle'))

  # print(img_tags)

  img_urls = []

  for img_tag in img_tags:
    url = img_tag.get("href")
    if url != None:
      img_urls.append(url)

  # print(img_urls)  # ダウンロードする写真データのurl

  m=150 # スクレイピングする画像数
  for fur_urls in img_urls:
    fr=requests.get(fur_urls)
    fur_soup=BeautifulSoup(fr.text, 'html.parser')
    fur_tags=fur_soup.find_all('img',  class_='view-product-image-print visible-print-block')

    for fur_tag in fur_tags:      
      fur_url=fur_tag.get("src")
      break
    
    if fur_url != None:
      furniture.append(fur_url)

    if len(furniture)==m:
      break

  print(len(furniture))

  if len(furniture)==m:
     break

In [ ]:
def download_image(url, file_path):
  r = requests.get(url, stream=True)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

In [ ]:
# ダウンロード先ファイルの参照(google drive用)
import os
from google.colab import drive
drive.mount('./gdrive')

google_drive_save_dir = "./gdrive/My Drive/Kansei/chair_concatenate/Polyester"

for index, url in enumerate(furniture):
  # if (index+1)%6==0:
  #   google_drive_save_dir = "./gdrive/My Drive/Kansei/chair_material_v2/val/Chenille"
  # elif (index+1)%6==1:
  #   google_drive_save_dir = "./gdrive/My Drive/Kansei/chair_material_v2/test/Chenille"
  # else:
  #   google_drive_save_dir = "./gdrive/My Drive/Kansei/chair_material_v2/train/Chenille"
    
  file_name = "{}.jpg".format(index+1)

  print(file_name)
  image_path = os.path.join(google_drive_save_dir, file_name)
  print(image_path)

  download_image(url=url, file_path=image_path)

In [ ]:
#valにコピーを保存
import glob
import os
import shutil
# from google.colab import drive
# drive.mount('./gdrive')

for i in range(1, 121):

  google_drive_save_dir = './gdrive/My Drive/Kansei/Armchairs_and_Accent_Chairs/train/Barrel_Chairs/%s.jpg'%(i*5)

  files = glob.glob(google_drive_save_dir)

  for file in files:
      try:
          new_file_path = file.replace('/train/Barrel_Chairs/', '/val/Barrel_Chairs/')
          shutil.move(file, new_file_path)

          # new_file_path = '/val/Wingback_Chairs/'
          # shutil.move(file, new_file_path)


      except FileNotFoundError:
          pass

      except OSError:
          pass